In [22]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import joblib
from math import exp

In [2]:
dataframe = pd.read_csv("dataset_final.csv",parse_dates=True,index_col='time_index')

In [4]:
features = pd.read_csv('autoencoders_features.csv').features

In [5]:
scaler = StandardScaler()
tags_dataframe = scaler.fit_transform(dataframe[features])

#saving scaling object
filename = 'scalar_object.sav'
joblib.dump(scaler, filename)

model = LogisticRegression(class_weight = 'balanced',n_jobs=-1,penalty='l2')
model.fit(tags_dataframe,dataframe.event)

filename = 'logistic_model.sav'
joblib.dump(model, filename)


C:\Users\BB-8\AppData\Roaming\Python\Python36\site-packages\sklearn\linear_model\logistic.py:1228: UserWarning: 'n_jobs' > 1 does not have any effect when 'solver' is set to 'liblinear'. Got 'n_jobs' = -1.
  " = {}.".format(self.n_jobs))


['logistic_model.sav']

In [33]:
df_feature_imp = pd.DataFrame()
df_feature_imp['features'] = features
df_feature_imp['imp'] = model.coef_.tolist()[0]
df_feature_imp.index = features
df_feature_imp.drop(['features'],axis=1,inplace=True)

In [34]:
df_feature_imp.to_csv('logistic_model_coefficient.csv')

In [35]:
df_coef = pd.read_csv('logistic_model_coefficient.csv',index_col='features')

In [58]:
tags_dataframe_prediction = scaler.transform(dataframe[features].tail(1))
dict_flow_indicators = dict()
logistic_equation = df_coef.imp * tags_dataframe_prediction[0]

In [59]:
summarize_equation = logistic_equation.sum()
tags_probability = 1 / (1 + exp(-summarize_equation))  

In [69]:
if tags_probability > 0.5:

    top_three_controllers_flow_indicator =logistic_equation.abs().sort_values(ascending=False).head(10).index.tolist()

    for i in top_three_controllers_flow_indicator:
        if df_coef.loc[i].values[0]>0:
            dict_flow_indicators[i]='Down'
        else:
            dict_flow_indicators[i]='Up'
else:
    print("None")

In [70]:
dict_flow_indicators

{'Line_12_Current_L4_W20_mean': 'Up',
 'Line_12_Current_L4_W30_mean': 'Up',
 'Line_12_Current_L4_W5_mean': 'Down',
 'Line_12_Temp_L4_W10_mean': 'Down',
 'Line_12_Temp_L4_W5_mean': 'Up',
 'Line_12_Velocity_W10_mean': 'Down',
 'Line_12_Velocity_W15_mean': 'Down',
 'Line_12_Velocity_W20_mean': 'Down',
 'Line_12_Velocity_W30_mean': 'Down',
 'Line_12_Velocity_W5_mean': 'Up'}